In [2]:
import numpy as np
import pandas as pd
import psycopg2

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.style
matplotlib.style.use('ggplot')
%matplotlib inline

In [3]:
# create a database connection
sqluser = 'mimic'
dbname = 'mimic'
schema_name = 'mimiciii'

cur = None

In [57]:
if cur: 
    cur.close()
    con.close()

con = psycopg2.connect(dbname = dbname, user = sqluser)
cur = con.cursor()
cur.execute('SET search_path to ' + schema_name)

In [4]:
query = \
"""
SELECT * FROM echo_features; 
"""
ef = pd.read_sql_query(query, con)
ef.head()

,row_id,icustay_id,hadm_id,subject_id,age_at_intime,gender,height,weight,bmi,ethnicity,...,mech_vent,fl_day1_input_ml,fl_day1_output_ml,fl_day1_balance_ml,fl_day2_input_ml,fl_day2_output_ml,fl_day2_balance_ml,fl_day3_input_ml,fl_day3_output_ml,fl_day3_balance_ml
0,103176,204988,100068,9889,27132 days 10:53:25,F,60.0,140.0,27.338889,BLACK/AFRICAN AMERICAN,...,1,291.415840,610.0,-318.584160,510.775198,827.0,-316.224802,440.015309,800.0,-359.984691
1,86108,272965,100282,30129,27235 days 17:33:37,F,NaN,160.0,NaN,UNABLE TO OBTAIN,...,1,11595.000000,1390.0,10205.000000,2914.000000,1715.0,1199.000000,1361.000000,1285.0,76.000000
2,86107,272965,100282,30129,27235 days 17:33:37,F,67.0,160.0,25.056806,UNABLE TO OBTAIN,...,1,640.000000,1578.0,-938.000000,240.000000,4170.0,-3930.000000,240.000000,2275.0,-2035.000000
3,99138,287636,100452,4410,-104004 days +21:31:40.290448,F,64.0,130.0,22.312012,BLACK/AFRICAN AMERICAN,...,0,795.000000,492.0,303.000000,120.000000,930.0,-810.000000,NaN,NaN,NaN
4,84813,230076,100465,91327,15504 days 18:06:11,F,65.0,165.0,27.454438,WHITE,...,0,231.333331,400.0,-168.666669,1090.000005,2840.0,-1749.999995,356.333332,1600.0,-1243.666668


In [12]:
filt = ef['ps_vaso']
print(len(ef.loc[filt, 'icustay_id'].unique()))
print(len(ef.loc[filt, 'hadm_id'].unique()))
print(len(ef.loc[filt, 'subject_id'].unique()))
ef.loc[filt].shape

8425
8318
7897


(11658, 112)

In [13]:
filt = ef['ps_vaso'] & ~ef['diag_xc']
print(len(ef.loc[filt, 'icustay_id'].unique()))
print(len(ef.loc[filt, 'hadm_id'].unique()))
print(len(ef.loc[filt, 'subject_id'].unique()))
ef.loc[filt].shape

2681
2667
2624


(3211, 112)

In [14]:
filt = ef['ps_vaso'] & ~ef['diag_xc'] & ~ef['chronic_dialysis_flg']
print(len(ef.loc[filt, 'icustay_id'].unique()))
print(len(ef.loc[filt, 'hadm_id'].unique()))
print(len(ef.loc[filt, 'subject_id'].unique()))
ef.loc[filt].shape

2548
2535
2499


(3041, 112)

In [15]:
filt = ef['ps_vaso'] & ~ef['diag_xc'] & ~ef['chronic_dialysis_flg'] & ef['age_filter']
print(len(ef.loc[filt, 'icustay_id'].unique()))
print(len(ef.loc[filt, 'hadm_id'].unique()))
print(len(ef.loc[filt, 'subject_id'].unique()))
ef.loc[filt].shape

2502
2489
2453


(2911, 112)

In [16]:
filt = ef['ps_vaso'] & ~ef['diag_xc'] & ~ef['chronic_dialysis_flg'] & ef['age_filter'] & ef['time_filter']
print(len(ef.loc[filt, 'icustay_id'].unique()))
print(len(ef.loc[filt, 'hadm_id'].unique()))
print(len(ef.loc[filt, 'subject_id'].unique()))
ef.loc[filt].shape

2117
2114
2094


(2239, 112)

# Calculate the number of echos filtered out by various diagnoses. 

In [36]:
query = \
"""
SELECT * FROM d_diagnoses_xc_annot 
WHERE exclude = 1
"""
dx = pd.read_sql_query(query, con)
dx.head()

,icd9_code,short_title,long_title,k_card,k_heart,k_hemorrhag,k_bleed,k_embolism,k_shock,k_clot,num,exclude,comments
0,3641,Meningococc pericarditis,Meningococcal pericarditis,1,0,0,0,0,0,0,11,1,None
1,521,Varicella pneumonitis,Varicella (hemorrhagic) pneumonitis,0,0,1,0,0,0,0,4,1,None
2,860,Chagas disease of heart,Chagas' disease with heart involvement,0,1,0,0,0,0,0,1,1,None
3,11281,Candidal endocarditis,Candidal endocarditis,1,0,0,0,0,0,0,6,1,None
4,1510,Mal neo stomach cardia,Malignant neoplasm of cardia,1,0,0,0,0,0,0,21,1,None


In [64]:
query = \
"""
SELECT pv.diag_xc IS NOT NULL AS diag_xc, ei.row_id, ei.icustay_id, ei.hadm_id, ei.subject_id FROM (
SELECT DISTINCT di.hadm_id
    , 1 as diag_xc
FROM d_diagnoses_xc_annot dx
INNER JOIN diagnoses_icd di
    ON di.icd9_code = dx.icd9_code
WHERE dx.exclude = 1) pv
RIGHT JOIN echo_icustay ei
    ON ei.hadm_id = pv.hadm_id
"""
res = pd.read_sql_query(query, con)
res.head()

,diag_xc,row_id,icustay_id,hadm_id,subject_id
0,True,104824,276795,124493,69851
1,True,59659,217810,188268,82208
2,True,67634,263932,168160,24182
3,True,99717,210970,199751,84020
4,True,74840,252250,127064,24477


Number of echos and subject ids with full diagnosis exclusions. 

In [65]:
print(len(res.loc[~res['diag_xc'], 'row_id'].unique()))
print(len(res.loc[~res['diag_xc'], 'hadm_id'].unique()))
print(len(res.loc[~res['diag_xc'], 'subject_id'].unique()))

6076
5119
4937


Exclude congestive heart failure

In [67]:
query = \
"""
SELECT pv.diag_xc IS NOT NULL AS diag_xc, ei.row_id, ei.icustay_id, ei.hadm_id, ei.subject_id FROM (
SELECT DISTINCT di.hadm_id
    , 1 as diag_xc
FROM d_diagnoses_xc_annot dx
INNER JOIN diagnoses_icd di
    ON di.icd9_code = dx.icd9_code
WHERE dx.exclude = 1 AND di.icd9_code <> '4280') pv
RIGHT JOIN echo_icustay ei
    ON ei.hadm_id = pv.hadm_id
"""
res = pd.read_sql_query(query, con)
res.head()

,diag_xc,row_id,icustay_id,hadm_id,subject_id
0,True,104824,276795,124493,69851
1,True,59659,217810,188268,82208
2,True,67634,263932,168160,24182
3,True,99717,210970,199751,84020
4,True,74840,252250,127064,24477


Only increased slightly by about 1000. 

In [68]:
print(len(res.loc[~res['diag_xc'], 'row_id'].unique()))
print(len(res.loc[~res['diag_xc'], 'hadm_id'].unique()))
print(len(res.loc[~res['diag_xc'], 'subject_id'].unique()))

7146
6016
5776


The reason for this is that it turns out that most patients who have congestive heart failure also has many other diagnoses in the excluded list. Try finding them. 

In [69]:
query = \
"""
WITH chf AS ( -- patients with congestive heart failure
    SELECT DISTINCT di.hadm_id
    FROM d_diagnoses_xc_annot dx
    INNER JOIN diagnoses_icd di
        ON di.icd9_code = dx.icd9_code
    INNER JOIN echo_icustay ei
        ON ei.hadm_id = di.hadm_id
    WHERE di.icd9_code = '4280'
) 
SELECT di.*, dx.short_title
FROM d_diagnoses_xc_annot dx
INNER JOIN diagnoses_icd di
    ON di.icd9_code = dx.icd9_code
INNER JOIN chf 
    ON chf.hadm_id = di.hadm_id
WHERE dx.exclude = 1
"""
res = pd.read_sql_query(query, con)
res.head()

,row_id,subject_id,hadm_id,seq_num,icd9_code,short_title
0,43746,3928,153920,3,4280,CHF NOS
1,516678,69776,126824,18,V433,Heart valve replac NEC
2,516671,69776,126824,11,4280,CHF NOS
3,516664,69776,126824,4,42833,Ac on chr diast hrt fail
4,49392,4423,121780,3,4280,CHF NOS


In [70]:
res.groupby('hadm_id')['icd9_code'].count().value_counts().sort_index()

1      897
2     2295
3     1976
4     1076
5      475
6      181
7       77
8       34
9        8
10       1
11       1
Name: icd9_code, dtype: int64

In [71]:
5119 + 897

6016